# Sample Notebook using the latest version of the SDK (v1.0.0-beta4)

## Step 1: Load required libraries

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 1.0.0-beta4"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta4 Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 1.0.0-beta4 Microsoft.SemanticKernel.Plugins.Core, 1.0.0-beta4 Microsoft.SemanticKernel.Plugins.Memory, 1.0.0-beta4 Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta4

In [2]:
// Enable the display of HTML content in the notebook
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

## Step 2: Setup Kernel

Import files from the `config` directory

In [3]:
#!import ../../../config/dotnet/StringHelpers.cs
#!import ../../../config/dotnet/ConfigModels.cs
#!import ../../../config/dotnet/ConfigHelper.cs
#!import ../../../config/dotnet/KernelHelper.cs

Display the current configuration (Optional).
> **⚠️ Note ☠️:** This is useful to check the current configuration. But be sure to comment this out & clear the output before committing the notebook. You don't want to commit your credentials. ⚠️


In [4]:
// Note: Comment this out & clear the output before committing the notebook. 
// You don't want to commit your credentials. 
// new ConfigurationHelper().GetConfiguration().Display();

Load the kernel

In [5]:
var kernel = KernelHelper.LoadKernel();
kernel.Display();

LoggerFactory
Functions
PromptTemplateEngine
HttpHandlerFactory
Memory
Skills


Display all available functions (via enabled plugins)

In [6]:
Microsoft.DotNet.Interactive.Formatting.Formatter.ListExpansionLimit = 100;
kernel.Functions.GetFunctionViews()
    .Select(fv => new { fv.PluginName, FunctionName=fv.Name, fv.Description })  
    .OrderBy(fv => fv.PluginName)
    .ThenBy(fv => fv.FunctionName)      
    .DisplayTable<dynamic>();

PluginName,FunctionName,Description
bing,Search,Perform a web search.
conversation,GetConversationActionItems,"Given a long conversation transcript, identify action items."
conversation,GetConversationTopics,"Given a long conversation transcript, identify topics worth remembering."
conversation,SummarizeConversation,"Given a long conversation transcript, summarize the conversation."
ConversationSummaryPlugin,funcb8580222f6c740f8ab590b8888ee6789,Analyze a conversation transcript and extract key topics worth remembering.
ConversationSummaryPlugin,funcbee9dfea6f634659969322ad6b640d58,"Given a section of a conversation transcript, summarize the part of the conversation."
ConversationSummaryPlugin,funccf6d1997b53a40ab87f47a253e05fdb2,"Given a section of a conversation transcript, identify action items."
file,Read,Read a file
file,Write,Write a file
http,Delete,Makes a DELETE request to a uri


## Step 3: Test Completions by creating a simple inline function

In [7]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using  Microsoft.DotNet.Interactive.Formatting;
// 1. define the function called "ask"
var askFunction = kernel.CreateSemanticFunction(
    promptTemplate: "{{$input}}", 
    requestSettings: new OpenAIRequestSettings() { 
        // The maximum number of tokens to generate in the completion.
        MaxTokens = 100,    
        // Temperature controls the randomness of the completion. The higher the temperature, the more random the completion.
        Temperature = 0.5,  
        // TopP controls the diversity of the completion. The higher the TopP, the more diverse the completion.
        TopP = 0.5,           
        // One or more sequences where the API will stop generating further tokens.
        StopSequences = new string[] { "\n" }, 
        // Here I have specified the service id. 
        // Comment this out if you would like to use the default service id.
        // ServiceId = "oai-gpt-4-0613" 
    },
    functionName: "ask");

// 2. call the function directly using the function object
KernelHelper.Print(await kernel
    .RunAsync(@"Why is the sky blue?", askFunction));

// ALTERNATIVELY
// 2. call the function by name using the kernel.
KernelHelper.Print(await kernel
    .Functions.GetFunction("ask").InvokeAsync(@"why is the ocean green?", kernel));

The sky appears blue because of a process called Rayleigh scattering. As sunlight reaches Earth's atmosphere, it is made up of different colors, which are seen as different wavelengths. Shorter wavelengths (like blue and violet) are scattered in all directions more than other colors like red, yellow, and green, which have longer wavelengths.

The color of the ocean can range from deep blue to green, depending on various factors. The primary reason for the ocean appearing green is the presence of algae and other plant life. When sunlight penetrates the ocean surface, it can cause photosynthesis in these plants, leading to a bloom of algae and phytoplankton, which can give the ocean a greenish hue.

## Step 4: Test Core Plugins

4.1: Text plugin

In [8]:
// Trim
KernelHelper.Print(await kernel
    .Functions
    .GetFunction("text", "trim")
    .InvokeAsync("  hello world  ", kernel)
, PrintElement.Pre);

// Trim Start
KernelHelper.Print(await kernel
    .Functions
    .GetFunction("text", "trimStart")
    .InvokeAsync("  hello world  ", kernel)
, PrintElement.Pre);

// Trim End
KernelHelper.Print(await kernel
    .Functions
    .GetFunction("text", "trimEnd")
    .InvokeAsync("  hello world  ", kernel)
, PrintElement.Pre);

hello world

hello world

hello world

4.1: This is a continuation of the previous step. Here we see how to "chain" plugins together. We call the "ask" plugin & pass the output to the "text" plugin where we can see the output in uppercase.

In [9]:
// Here we demonstrate chaining functions together
// First we ask the question using the ask function (which we wrote earlier),
// then we uppercase the response using the text.uppercase function (part of the core plugins).
KernelHelper.Print(await kernel.RunAsync(
    @"Tell me 1 sentence joke.",
    kernel.Functions.GetFunction("ask"),
    kernel.Functions.GetFunction("text", "uppercase")));

WHY DON'T SCIENTISTS TRUST ATOMS? BECAUSE THEY MAKE UP EVERYTHING!

## Step 5: Test Web Plugins

In [10]:
// Here we use the bing plugin (part of the web plugins) to search the web for the query "What is the inflation rate in the US in November 2023?"
// In addition we use funcion chaining to summarize the conversation. (part of the core plugins).
KernelHelper.Print(await kernel.RunAsync(
    @"What is the inflation rate in the Australia in November 2023?",
    kernel.Functions.GetFunction("bing", "search"),
    kernel.Functions.GetFunction("conversation", "SummarizeConversation")));

The content discusses the annual inflation rates in Australia from 1950, with a focus on recent data up to September 2023. The Consumer Price Index (CPI) in Australia rose by 1.2% in the third quarter of 2023. This inflation rebound was driven by surging fuel prices and has implications for Australians with mortgages. The economy has shown resilience despite a significant increase in interest rates, marking the second steepest monetary policy tightening cycle in the bank's history. However, it's noted that consumers are cutting back in response. There's also a prediction that interest rates are likely to rise further. The highest inflation rate month-over-month (MoM) in Australia was 7.55% in the fourth quarter of 1951, and the lowest was -1.90% in the second quarter of 2020.

## Step 6: Test Memories

In [12]:
var db = "random-animals";
var ids = new List<string>();

// Save some information to the database
ids.Add(await KernelHelper.SaveChunkAsync(
    kernel: kernel, db: db, text: "Dog"));
ids.Add(await KernelHelper.SaveChunkAsync(
    kernel: kernel, db: db, text: "Elephant"));
ids.Add(await KernelHelper.SaveChunkAsync(
    kernel: kernel, db: db, text: "Octopus"));
KernelHelper.Info($"Saved {ids.Count} items to the {db} collection.");

// Search the database
var query = @"Has tentacles?";
var rs = await KernelHelper.SearchDatabaseAsync(
    kernel: kernel, 
    db: db, 
    q: query, 
    limit: 1, 
    minRelevanceScore: 0.5
);
KernelHelper.Print($"{query} - {rs.FirstOrDefault()?.Text}");

// Cleanup the database
var cleanup = await KernelHelper.DeleteChunkAsync(
    kernel: kernel, 
    db: db, 
    ids: ids);
KernelHelper.Info($"Deleted {cleanup.Count} items from the {db} collection.");

Saved 3 items to the random-animals collection.

Has tentacles? - Octopus

Deleted 3 items from the random-animals collection.

Alternatively, chunk and store a whole book!

In [13]:
// Read file & convert it into smaller chunks
var db = "intelligent-investor";
var ids = await KernelHelper.SaveFileAsync(
    kernel: kernel, 
    db: db, 
    path: "../../../data/intelligent-investor.txt");
KernelHelper.Info($"Saved {ids.Count} items to the {db} collection.");

- Saved chunk 0.
- Saved chunk 1.
- Saved chunk 2.
- Saved chunk 3.
- Saved chunk 4.
- Saved chunk 5.
- Saved chunk 6.
- Saved chunk 7.
- Saved chunk 8.
- Saved chunk 9.
- Saved chunk 10.
- Saved chunk 11.
- Saved chunk 12.
- Saved chunk 13.
- Saved chunk 14.
- Saved chunk 15.
- Saved chunk 16.
- Saved chunk 17.
- Saved chunk 18.
- Saved chunk 19.
- Saved chunk 20.
- Saved chunk 21.
- Saved chunk 22.
- Saved chunk 23.
- Saved chunk 24.
- Saved chunk 25.
- Saved chunk 26.
- Saved chunk 27.
- Saved chunk 28.
- Saved chunk 29.
- Saved chunk 30.
- Saved chunk 31.
- Saved chunk 32.
- Saved chunk 33.
- Saved chunk 34.
- Saved chunk 35.
- Saved chunk 36.
- Saved chunk 37.
- Saved chunk 38.
- Saved chunk 39.
- Saved chunk 40.
- Saved chunk 41.
- Saved chunk 42.
- Saved chunk 43.
- Saved chunk 44.
- Saved chunk 45.


Saved 46 items to the intelligent-investor collection.

In [14]:
// Search the database
var query = @"Ben Graham's investment philosophy?";
KernelHelper.Print(query, PrintElement.Header);
var rs = await KernelHelper.SearchDatabaseAsync(
    kernel: kernel, 
    db: db, 
    q: query, 
    limit: 2, 
    minRelevanceScore: 0.5
);
rs.ForEach(r => query += $"\n\n{r.Text}");
//KernelHelper.Print($"{query} \n {rs.FirstOrDefault()?.Text}");
KernelHelper.Print(await kernel.RunAsync(
    query,
    kernel.Functions.GetFunction("conversation", "SummarizeConversation")));

Ben Graham's investment philosophy?

Ben Graham's investment philosophy emphasizes the dangers of projection due to the uncertainty of the future, including factors like inflation, economic recessions, pandemics, and geopolitical upheavals. He believes that trying to time the market is a futile effort for an ordinary investor. Instead, Graham advocates for a protection-based approach that doesn't attempt to time the market. He suggests that value investors should focus on identifying and investing in large, conservatively financed companies whose present value, estimated by tangible assets, is significantly lower than their current stock prices. His book, "The Intelligent Investor," provides guidance on developing the mindset necessary to avoid panic during market fluctuations. Graham also differentiates between two types of investors: defensive investors, who aim to protect their capital, generate decent returns, and minimize frequent decisions, and enterprising investors, who actively manage their portfolios, investing more time in stock selection without necessarily taking on more risk.

In [15]:
// Cleanup the database
var cleanup = await KernelHelper.DeleteChunkAsync(
    kernel: kernel, 
    db: db, 
    ids: ids);
KernelHelper.Info($"Deleted {cleanup.Count} items from the {db} collection.");

Deleted 46 items from the intelligent-investor collection.